# ES → BBF Billing Account Migration (v8)

This notebook creates new BBF billing accounts for ES accounts that have orders moving to BBF.

## Process Overview
1. Query all `billing_invoice__c` with active orders
2. Classify each order:
   - **WD:** PA market AND `Project_Group__c` contains 'PA MARKET DECOM' → Stays on current BAN
   - **BBF:** Everything else → Moves to new -BBF account
3. Create accounts based on mix:
   - **All orders → WD** → **SKIP** (no new account - orders stay in place)
   - **All orders → BBF** → Create 1 account with `-BBF` suffix
   - **Mixed** → Create 1 `-BBF` account (WD orders stay on original BAN)
4. Set `legacy_es_id__c` to link back to original billing invoice
5. Output mapping to Excel

## v8 Changes from v7
- **No longer creating -WD accounts**
- PA MARKET DECOM orders stay on their existing Billing_Invoice__c
- Only create -BBF accounts for BANs that have BBF orders
- BANs that are 100% PA MARKET DECOM are skipped entirely

## Classification Logic
| BAN Type | Action |
|----------|--------|
| WD-only (all PA MARKET DECOM) | SKIP - no new account |
| BBF-only (no PA MARKET DECOM) | Create 1 -BBF account |
| Mixed | Create 1 -BBF account |

## Safety
- `DRY_RUN = True` by default
- Dry run outputs Excel showing what would be created
- Live run creates accounts and outputs mapping

In [12]:
# === SETUP ===
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from collections import defaultdict
import time
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Entity mapping (from original sync)
entity_map = {
    "Everstream Ohio": "EVO",
    "Everstream Michigan": "EVM",
    "OneCommunity": "EVO",
    "Medina County Fiber Network Services": "OCO",
    "Everstream Lynx": "EVL",
    "Michigan-LB": "EVB",
    "Michigan-RF": "EVR",
    "Everstream PA": "EVP",
}

# PA Markets
PA_MARKETS = ["Pittsburgh", "Harrisburg", "Philadelphia", "Scranton", "Uniti-PA"]

# WD Classification: PA market + this string in Project_Group__c
WD_PROJECT_GROUP = "PA MARKET DECOM"

# Connection credentials
heroku_conn_kwargs = {
    "dbname": "d88otjf7uhv9pr",
    "user": "ucn7cbk14sd6h",
    "password": "pf27d102f95e996e621e02523d035a1bff27590c8e6a13f5b180703a6631320c5",
    "host": "ec2-54-86-217-174.compute-1.amazonaws.com",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

oss_conn_kwargs = {
    "dbname": "GLC",
    "user": "oss_server",
    "password": "3wU3uB28X?!r2?@ebrUg",
    "host": "pg01.comlink.net",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

print("Connecting to Heroku...")
conn = psycopg2.connect(**heroku_conn_kwargs)
print("✅ Connected to Heroku")

print("Connecting to OSS...")
oconn = psycopg2.connect(**oss_conn_kwargs)
print("✅ Connected to OSS")

Connecting to Heroku...
✅ Connected to Heroku
Connecting to OSS...
✅ Connected to OSS


In [13]:
# === CONFIGURATION ===
DRY_RUN = True  # Set to False to actually create accounts
OUTPUT_DIR = "."  # Change to your preferred output directory

# Output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_type = "dry_run" if DRY_RUN else "live_run"
output_file = f"{OUTPUT_DIR}/es_bbf_migration_v8_{run_type}_{timestamp}.xlsx"

print(f"DRY_RUN: {DRY_RUN}")
print(f"Output file: {output_file}")

DRY_RUN: True
Output file: ./es_bbf_migration_v8_dry_run_20260106_074822.xlsx


In [14]:
# === STEP 1: Get all billing_invoice__c with active services and order classification ===

# Build PA markets string for SQL
pa_markets_str = "','".join(PA_MARKETS)

# v8 FILTERS:
# - Status: Activated, Disconnect in Progress
# - WD = PA market + Project Group contains 'PA MARKET DECOM'
# - BBF = Everything else

active_billing_accounts_sql = f"""
SELECT 
    bi.account_number__c AS old_account_id,
    bi.sfid AS old_billing_invoice_sfid,
    bi.account__c AS customer_sfid,
    bi.name AS old_ban_name,
    bi.account_name__c,
    bi.billing_address_1__c,
    bi.billing_city__c,
    bi.billing_state__c,
    bi.billing_zip__c,
    bi.billing_e_mail__c,
    bi.late_fee_percentage__c,
    bi.disable_late_fees__c,
    bi.payment_terms__c,
    bi.invoice_delivery_preference__c,
    COUNT(DISTINCT o.sfid) AS total_active_orders,
    -- WD orders: PA market AND Project_Group__c contains 'PA MARKET DECOM'
    COUNT(DISTINCT CASE 
        WHEN o.dimension_4_market__c IN ('{pa_markets_str}') 
             AND o.project_group__c LIKE '%{WD_PROJECT_GROUP}%'
        THEN o.sfid 
    END) AS wd_order_count,
    -- BBF orders: Everything else
    COUNT(DISTINCT CASE 
        WHEN NOT (
            o.dimension_4_market__c IN ('{pa_markets_str}') 
            AND o.project_group__c LIKE '%{WD_PROJECT_GROUP}%'
        )
        THEN o.sfid 
    END) AS bbf_order_count
FROM sfprod.billing_invoice__c bi
JOIN sfprod."order" o ON o.billing_invoice__c = bi.sfid
WHERE 
    -- Status filter: Activated, Disconnect in Progress
    o.status IN ('Activated', 'Disconnect in Progress')
    -- Standard filters
    AND bi.disabled__c = false
    AND bi.account_number__c IS NOT NULL
GROUP BY 
    bi.account_number__c, bi.sfid, bi.account__c, bi.name, bi.account_name__c,
    bi.billing_address_1__c, bi.billing_city__c, bi.billing_state__c, 
    bi.billing_zip__c, bi.billing_e_mail__c, bi.late_fee_percentage__c, 
    bi.disable_late_fees__c, bi.payment_terms__c, bi.invoice_delivery_preference__c
ORDER BY bi.account_number__c;
"""

print("Executing query (this may take a moment)...")
with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(active_billing_accounts_sql)
    active_billing_accounts = cur.fetchall()

# Categorize by destination mix
wd_only = [
    ba
    for ba in active_billing_accounts
    if ba["wd_order_count"] > 0 and ba["bbf_order_count"] == 0
]
bbf_only = [
    ba
    for ba in active_billing_accounts
    if ba["bbf_order_count"] > 0 and ba["wd_order_count"] == 0
]
mixed = [
    ba
    for ba in active_billing_accounts
    if ba["wd_order_count"] > 0 and ba["bbf_order_count"] > 0
]

# v8: Only create accounts for BANs with BBF orders
bans_needing_bbf_account = bbf_only + mixed

print(
    f"\nFound {len(active_billing_accounts)} ES billing accounts with active services:"
)
print(f"  - WD only (PA MARKET DECOM): {len(wd_only)} → SKIP (no new account needed)")
print(f"  - BBF only: {len(bbf_only)} → Create {len(bbf_only)} -BBF accounts")
print(f"  - Mixed: {len(mixed)} → Create {len(mixed)} -BBF accounts")
print(f"\n✅ Total -BBF accounts to create: {len(bans_needing_bbf_account)}")
print(f"⏭️  Skipping {len(wd_only)} WD-only BANs (orders stay on current BAN)")

Executing query (this may take a moment)...

Found 2519 ES billing accounts with active services:
  - WD only (PA MARKET DECOM): 17 → SKIP (no new account needed)
  - BBF only: 2487 → Create 2487 -BBF accounts
  - Mixed: 13 → Create 13 -BBF accounts

✅ Total -BBF accounts to create: 2500
⏭️  Skipping 17 WD-only BANs (orders stay on current BAN)


In [15]:
# === STEP 2: Get SF Account data for each billing account ===

# Only get customer SFIDs for BANs that need BBF accounts
customer_sfids = list(
    set([ba["customer_sfid"] for ba in bans_needing_bbf_account if ba["customer_sfid"]])
)
customer_sfids_str = "','".join(customer_sfids)

sf_accounts_sql = f"""
SELECT 
    sfid,
    name,
    billingstreet,
    billingcity,
    billingstate,
    billingpostalcode,
    email_address__c,
    onecommunity_entity__c,
    business_sector__c,
    send_consolidated_invoice__c
FROM sfprod.account
WHERE sfid IN ('{customer_sfids_str}')
"""

with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(sf_accounts_sql)
    sf_accounts = {row["sfid"]: row for row in cur.fetchall()}

print(f"Retrieved {len(sf_accounts)} SF Account records.")

Retrieved 2253 SF Account records.


In [16]:
# === STEP 3: Get OSS Customer records ===

oss_customers_sql = f"""
SELECT 
    customer_id,
    customer_nm,
    salesforce_id
FROM customers.customers
WHERE salesforce_id IN ('{customer_sfids_str}')
  AND disabled >= now()
"""

with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
    ocur.execute(oss_customers_sql)
    oss_customers = {row["salesforce_id"]: row for row in ocur.fetchall()}

print(f"Retrieved {len(oss_customers)} OSS Customer records.")

# Check for missing customers
missing_customers = [sfid for sfid in customer_sfids if sfid not in oss_customers]
if missing_customers:
    print(
        f"⚠️ Warning: {len(missing_customers)} SF Accounts have no OSS Customer record"
    )
else:
    print("✅ All SF Accounts have matching OSS Customer records!")

Retrieved 2252 OSS Customer records.
⚠️ Warning: 1 SF Accounts have no OSS Customer record


In [17]:
# === HELPER FUNCTIONS ===


def create_oss_account(sf_account_info, customer_id, ocur):
    """
    Creates a new OSS billing account using SF Account data.

    Args:
        sf_account_info: SF Account record
        customer_id: OSS customer_id
        ocur: OSS cursor

    Returns:
        (account_record, error_message)
    """
    # Parse zip code
    billing_zip = sf_account_info.get("billingpostalcode") or ""
    if "-" in billing_zip:
        zip_cd, zip4 = billing_zip.split("-", 1)
    else:
        zip_cd = billing_zip
        zip4 = None

    # Get entity code
    entity = sf_account_info.get("onecommunity_entity__c")
    if entity not in entity_map:
        return None, f"Unknown entity: {entity}"
    company_cd = entity_map[entity]

    # Determine customer type
    business_sector = sf_account_info.get("business_sector__c") or ""
    customer_type_cd = "W" if business_sector == "Wholesale" else "R"

    # Insert new account - bbf_ban is always True for v8
    ocur.execute(
        """
        INSERT INTO customers.accounts (
            account_nm,
            customer_id,
            company_cd,
            address1,
            city,
            state_cd,
            zip,
            zip4,
            billing_email,
            created_by_id,
            late_fee_percentage,
            customer_type_cd,
            bbf_ban
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, 0, %s, %s, %s
        ) RETURNING *
    """,
        (
            sf_account_info["name"],
            customer_id,
            company_cd,
            sf_account_info.get("billingstreet"),
            sf_account_info.get("billingcity"),
            sf_account_info.get("billingstate"),
            zip_cd,
            zip4,
            sf_account_info.get("email_address__c"),
            0.015,  # Default late fee percentage
            customer_type_cd,
            True,  # bbf_ban is always True for BBF accounts
        ),
    )

    return ocur.fetchone(), None


def create_sf_billing_account(
    oss_account_info, customer_sfid, legacy_billing_sfid, cur
):
    """
    Creates a new SF Billing Invoice record in Heroku.

    Args:
        oss_account_info: OSS account record
        customer_sfid: SF Account SFID
        legacy_billing_sfid: Original billing invoice SFID
        cur: Heroku cursor
    """
    # Build account name with -BBF suffix
    new_ban_name = f"A{oss_account_info['account_id']}-BBF"

    # Determine invoice delivery preference
    paperless = oss_account_info.get("paperless", False)
    email_non_paperless = oss_account_info.get("email_non_paperless", False)

    if paperless:
        delivery_pref = "E-mail"
    elif not paperless and email_non_paperless:
        delivery_pref = "E-mail & Paper"
    else:
        delivery_pref = "Paper"

    cur.execute(
        """
        INSERT INTO sfprod.billing_invoice__c (
            billing_city__c,
            billing_address_1__c,
            account__c,
            account_number__c,
            name,
            late_fee_percentage__c,
            billing_zip__c,
            billing_state__c,
            account_name__c,
            billing_address_2__c,
            disable_late_fees__c,
            suppress_invoice_generation__c,
            payment_terms__c,
            suppress_past_due_notifications__c,
            invoice_delivery_preference__c,
            billing_e_mail__c,
            bbf_ban__c,
            legacy_es_id__c
        ) VALUES (
            %(city)s,
            %(address1)s,
            %(customer_sfid)s,
            %(account_id)s,
            %(ban_name)s,
            %(late_fee_percentage)s,
            %(zip)s,
            %(state_cd)s,
            %(account_nm)s,
            %(address2)s,
            %(disable_late_fees)s,
            false,
            %(due_date_frequency_cd)s,
            %(no_past_due_notice)s,
            %(delivery_pref)s,
            %(billing_email)s,
            true,
            %(legacy_billing_sfid)s
        )
        ON CONFLICT (account_number__c) DO NOTHING
    """,
        {
            "city": oss_account_info.get("city"),
            "address1": oss_account_info.get("address1"),
            "customer_sfid": customer_sfid,
            "account_id": str(oss_account_info["account_id"]),
            "ban_name": new_ban_name,
            "late_fee_percentage": oss_account_info.get("late_fee_percentage", 0.015),
            "zip": oss_account_info.get("zip"),
            "state_cd": oss_account_info.get("state_cd"),
            "account_nm": oss_account_info.get("account_nm"),
            "address2": oss_account_info.get("address2"),
            "disable_late_fees": oss_account_info.get("disable_late_fees", False),
            "due_date_frequency_cd": oss_account_info.get(
                "due_date_frequency_cd", "NET30"
            ),
            "no_past_due_notice": oss_account_info.get("no_past_due_notice", False),
            "delivery_pref": delivery_pref,
            "billing_email": oss_account_info.get("billing_email"),
            "legacy_billing_sfid": legacy_billing_sfid,
        },
    )


def poll_for_sfid(account_id, customer_sfid, cur, max_retries=5, sleep_seconds=5):
    """
    Polls Heroku waiting for Salesforce to assign an SFID to the new billing invoice.
    """
    new_billing_acct = None
    retry_count = 0

    while retry_count <= max_retries and (
        new_billing_acct is None or new_billing_acct.get("sfid") is None
    ):
        retry_count += 1
        cur.execute(
            """
            SELECT sfid, account__c, account_number__c, name
            FROM sfprod.billing_invoice__c
            WHERE account__c = %s
              AND account_number__c = %s
        """,
            (customer_sfid, str(account_id)),
        )

        billing_acct = cur.fetchone()
        if billing_acct and billing_acct.get("sfid"):
            new_billing_acct = billing_acct
        else:
            time.sleep(sleep_seconds)

    return new_billing_acct

In [18]:
# === STEP 4: Process each billing account (BBF accounts only) ===

results = []
skipped_wd_only = []  # Track WD-only BANs that were skipped

# First, record the skipped WD-only BANs
for ba in wd_only:
    skipped_wd_only.append(
        {
            "old_account_id": ba["old_account_id"],
            "old_billing_invoice_sfid": ba["old_billing_invoice_sfid"],
            "account_name": ba["account_name__c"],
            "old_ban_name": ba["old_ban_name"],
            "wd_order_count": ba["wd_order_count"],
            "bbf_order_count": ba["bbf_order_count"],
            "reason": "WD-only: All orders are PA MARKET DECOM - stays on current BAN",
        }
    )

print(f"Recorded {len(skipped_wd_only)} WD-only BANs as skipped")

# Now process BANs that need BBF accounts
total = len(bans_needing_bbf_account)

for idx, ba in enumerate(bans_needing_bbf_account, 1):
    old_account_id = ba["old_account_id"]
    customer_sfid = ba["customer_sfid"]
    old_billing_sfid = ba["old_billing_invoice_sfid"]
    wd_count = ba["wd_order_count"]
    bbf_count = ba["bbf_order_count"]

    # Determine market type
    if bbf_count > 0 and wd_count == 0:
        market_type = "BBF_ONLY"
    else:
        market_type = "MIXED"

    # Get SF Account
    sf_account = sf_accounts.get(customer_sfid)
    if not sf_account:
        result = {
            "old_account_id": old_account_id,
            "old_billing_invoice_sfid": old_billing_sfid,
            "account_name": ba["account_name__c"],
            "old_ban_name": ba["old_ban_name"],
            "market_type": market_type,
            "wd_order_count": wd_count,
            "bbf_order_count": bbf_count,
            "new_account_id": None,
            "new_ban_name": None,
            "new_billing_invoice_sfid": None,
            "status": "SKIPPED",
            "error": "SF Account not found",
        }
        results.append(result)
        continue

    # Get OSS Customer
    oss_customer = oss_customers.get(customer_sfid)
    if not oss_customer:
        result = {
            "old_account_id": old_account_id,
            "old_billing_invoice_sfid": old_billing_sfid,
            "account_name": ba["account_name__c"],
            "old_ban_name": ba["old_ban_name"],
            "market_type": market_type,
            "wd_order_count": wd_count,
            "bbf_order_count": bbf_count,
            "new_account_id": None,
            "new_ban_name": None,
            "new_billing_invoice_sfid": None,
            "status": "SKIPPED",
            "error": "OSS Customer not found",
        }
        results.append(result)
        continue

    customer_id = oss_customer["customer_id"]

    result = {
        "old_account_id": old_account_id,
        "old_billing_invoice_sfid": old_billing_sfid,
        "account_name": ba["account_name__c"],
        "old_ban_name": ba["old_ban_name"],
        "market_type": market_type,
        "wd_order_count": wd_count,
        "bbf_order_count": bbf_count,
        "new_account_id": None,
        "new_ban_name": None,
        "new_billing_invoice_sfid": None,
        "status": None,
        "error": None,
    }

    # === DRY RUN ===
    if DRY_RUN:
        entity = sf_account.get("onecommunity_entity__c")
        if entity not in entity_map:
            result["status"] = "WOULD_SKIP"
            result["error"] = f"Unknown entity: {entity}"
        else:
            result["new_account_id"] = "(pending)"
            result["new_ban_name"] = "A(pending)-BBF"
            result["new_billing_invoice_sfid"] = "(pending)"
            result["status"] = "WOULD_CREATE"

        results.append(result)
        continue

    # === LIVE RUN ===
    try:
        print(
            f"[{idx}/{total}] Creating -BBF account for {ba['account_name__c']} (old: {old_account_id})"
        )

        # Step 1: Create OSS Account
        with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
            new_oss_account, error = create_oss_account(sf_account, customer_id, ocur)

            if error:
                result["status"] = "SKIPPED"
                result["error"] = error
                results.append(result)
                oconn.rollback()
                continue

            new_account_id = new_oss_account["account_id"]
            result["new_account_id"] = new_account_id
            result["new_ban_name"] = f"A{new_account_id}-BBF"

            # Commit OSS transaction
            oconn.commit()
            print(f"  ✅ Created OSS account: {new_account_id} (bbf_ban=True)")

        # Step 2: Create SF Billing Invoice in Heroku
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            create_sf_billing_account(
                new_oss_account, customer_sfid, old_billing_sfid, cur
            )
            conn.commit()
            print(f"  ✅ Created SF Billing Invoice: A{new_account_id}-BBF")

        # Step 3: Poll for SFID
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            new_billing_acct = poll_for_sfid(new_account_id, customer_sfid, cur)

            if new_billing_acct and new_billing_acct.get("sfid"):
                result["new_billing_invoice_sfid"] = new_billing_acct["sfid"]
                result["status"] = "CREATED"
                print(f"  ✅ Got SFID: {new_billing_acct['sfid']}")
            else:
                result["status"] = "CREATED_NO_SFID"
                result["error"] = "Timed out waiting for SFID"
                print(f"  ⚠️ Created but no SFID yet")

    except Exception as e:
        result["status"] = "ERROR"
        result["error"] = str(e)
        oconn.rollback()
        conn.rollback()
        print(f"  ❌ Error: {e}")

    results.append(result)

    if idx % 100 == 0:
        print(f"Processed {idx}/{total} billing accounts...")

# Summary
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)
status_counts = defaultdict(int)
for r in results:
    status_counts[r["status"]] += 1

for status, count in sorted(status_counts.items()):
    print(f"{status}: {count}")

bbf_acct_count = len(
    [
        r
        for r in results
        if r["status"] in ("WOULD_CREATE", "CREATED", "CREATED_NO_SFID")
    ]
)
print(f"\n-BBF accounts to create: {bbf_acct_count}")
print(f"WD-only BANs skipped: {len(skipped_wd_only)}")

Recorded 17 WD-only BANs as skipped

SUMMARY
SKIPPED: 1
WOULD_CREATE: 2499

-BBF accounts to create: 2499
WD-only BANs skipped: 17


In [ ]:
# === STEP 5: Create output Excel file ===

wb = Workbook()

# --- Sheet 1: BBF Account Clone Results ---
ws1 = wb.active
ws1.title = "BBF Clone Results"

# Styling
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Headers
headers = [
    "Old Account ID",
    "New Account ID",
    "Account Name",
    "Old BAN Name",
    "New BAN Name",
    "Market Type",
    "WD Orders (Stay)",
    "BBF Orders (Move)",
    "Old Billing SFID",
    "New Billing SFID",
    "Status",
    "Error",
]
ws1.append(headers)

for col, header in enumerate(headers, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Status colors
status_colors = {
    "WOULD_CREATE": "C6EFCE",  # Light green
    "WOULD_SKIP": "FCE4D6",  # Light orange
    "CREATED": "C6EFCE",  # Light green
    "CREATED_NO_SFID": "FFEB9C",  # Yellow
    "SKIPPED": "FFCCCC",  # Light red
    "ERROR": "FF6666",  # Red
}

# Data rows
for row_idx, r in enumerate(results, 2):
    row_data = [
        r["old_account_id"],
        r["new_account_id"],
        r["account_name"],
        r["old_ban_name"],
        r["new_ban_name"],
        r["market_type"],
        r["wd_order_count"],
        r["bbf_order_count"],
        r["old_billing_invoice_sfid"],
        r["new_billing_invoice_sfid"],
        r["status"],
        r["error"],
    ]
    ws1.append(row_data)

    # Color by status
    fill_color = status_colors.get(r["status"], "FFFFFF")
    for col in range(1, len(headers) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

# Auto-fit columns
for col in ws1.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    ws1.column_dimensions[column].width = min(max_length + 2, 40)

# Freeze header
ws1.freeze_panes = "A2"

# --- Sheet 2: Skipped WD-Only BANs ---
ws2 = wb.create_sheet("Skipped WD-Only BANs")
wd_headers = [
    "Old Account ID",
    "Account Name",
    "Old BAN Name",
    "WD Orders",
    "BBF Orders",
    "Old Billing SFID",
    "Reason",
]
ws2.append(wd_headers)
for col, header in enumerate(wd_headers, 1):
    cell = ws2.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = PatternFill("solid", fgColor="5B9BD5")  # Blue
    cell.alignment = header_alignment

for r in skipped_wd_only:
    ws2.append(
        [
            r["old_account_id"],
            r["account_name"],
            r["old_ban_name"],
            r["wd_order_count"],
            r["bbf_order_count"],
            r["old_billing_invoice_sfid"],
            r["reason"],
        ]
    )

ws2.freeze_panes = "A2"

# --- Sheet 3: Summary ---
ws3 = wb.create_sheet("Summary")
ws3.append(["ES → BBF Account Migration Summary (v8)"])
ws3["A1"].font = Font(bold=True, size=14)
ws3.append([])
ws3.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE RUN"])
ws3.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws3.append([])
ws3.append(["v8 Logic:"])
ws3["A6"].font = Font(bold=True)
ws3.append(
    ["  - WD-only BANs (all PA MARKET DECOM):", "SKIP - orders stay on current BAN"]
)
ws3.append(["  - BBF-only BANs:", "Create 1 -BBF account"])
ws3.append(["  - Mixed BANs:", "Create 1 -BBF account (WD orders stay on original)"])
ws3.append([])
ws3.append(["BAN Breakdown:"])
ws3["A11"].font = Font(bold=True)
ws3.append(["  WD-only (skipped):", len(wd_only)])
ws3.append(["  BBF-only:", len(bbf_only)])
ws3.append(["  Mixed:", len(mixed)])
ws3.append([])
ws3.append(["Accounts to Create:"])
ws3["A16"].font = Font(bold=True)
ws3.append(["  -BBF accounts:", bbf_acct_count])
ws3.append([])
ws3.append(["Status Counts:"])
ws3["A19"].font = Font(bold=True)
for status, count in sorted(status_counts.items()):
    ws3.append([f"  {status}:", count])
ws3.append([])
ws3.append(["PA Markets:", ", ".join(PA_MARKETS)])
ws3.append(["WD Project Group:", WD_PROJECT_GROUP])

# --- Sheet 4: Skipped/Errors ---
ws4 = wb.create_sheet("Errors")
error_headers = ["Old Account ID", "Account Name", "Market Type", "Status", "Error"]
ws4.append(error_headers)
for col, header in enumerate(error_headers, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = PatternFill("solid", fgColor="C00000")  # Red
    cell.alignment = header_alignment

for r in results:
    if r["status"] in ("SKIPPED", "WOULD_SKIP", "ERROR"):
        ws4.append(
            [
                r["old_account_id"],
                r["account_name"],
                r["market_type"],
                r["status"],
                r["error"],
            ]
        )

ws4.freeze_panes = "A2"

# Save
wb.save(output_file)
print(f"\n✅ Output saved to: {output_file}")

In [ ]:
# === STEP 6: Cleanup ===

if DRY_RUN:
    print("\n🔄 DRY RUN complete. No accounts were created.")
    print(f"Review the output file: {output_file}")
    print("\nTo run for real, set DRY_RUN = False and re-run the notebook.")
else:
    print(f"\n✅ LIVE RUN complete.")
    print(f"Created {bbf_acct_count} -BBF accounts.")
    print(f"Skipped {len(skipped_wd_only)} WD-only BANs.")
    print(f"Output saved to: {output_file}")

In [ ]:
# === Close connections ===
conn.close()
oconn.close()
print("🔌 Database connections closed.")

---
## Next Phase: Move Orders

After BBF accounts are created, use the order migration notebook to:
1. Move BBF orders → new `-BBF` accounts
2. PA MARKET DECOM orders stay on their original BANs

The `legacy_es_id__c` field links each new account back to its original.